In [44]:
from IPython.display import display
from IPython.display import Image
from ipywidgets import *

In [45]:
from PIL import Image

In [46]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [47]:
#hide Now we can grab the image
# For the book, we can't actually click an upload button, so we fake it

#upload_filename = list(args['new'].keys())[0]
#btn_upload = SimpleNamespace(data = ['upload_filename'])

In [48]:
#img = PILImage.create(btn_upload.data[-1])

In [56]:
# #hide_output / We can use an Output widget to display it
out_pl = widgets.Output()
# out_pl.clear_output()
# with out_pl: display(img.to_thumb(128,128))
# out_pl

In [57]:
# pred,pred_idx,probs = learn_inf.predict(img)

In [58]:
#hide_output / and use a Label to display them:
#lbl_pred = widgets.Label()
#lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
#lbl_pred

In [59]:
#hide_output / We'll need a button to do the classification. It looks exactly like the upload button:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [60]:
# We'll also need a click event handler; that is, a function that will be called when it's pressed. 
# We can just copy over the lines of code from above
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run = widgets.Button(description='Classify')
btn_run.on_click(on_click_classify)

In [61]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [62]:
#hide_output
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

NameError: name 'learn_inf' is not defined

In [ ]:
#hide
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila 

In [ ]:
# !jupyter nbextension list

In [ ]:
# ! jupyter serverextension list

In [ ]:
#voila plants_classifier.ipynb

In [ ]:
# 깃허브, binder

In [7]:
# from IPython.display import display
# from ipywidgets import widgets

# def upload_file(args):
#     upload_filename = list(args['new'].keys())[0]
#     uploaded_file = args['new'][upload_filename]
#     content = uploaded_file.get('content')
#     print(uploaded_file)
#     print(uploaded_file.get('metadata').get('name'))
    
# w = widgets.FileUpload(
#     accept='image/png, image/jpeg',
#     multiple=False 
# )

# display(w)
# w.observe(upload_file, names=['value'])

# #widgets.Image(value=content)

FileUpload(value={}, accept='image/png, image/jpeg', description='Upload')

In [ ]:
# 업로드된 이미지 모델에 사용

# 결과 출력 ( )

In [ ]:
# Voila, Binder 웹 배포 
# https://mybinder.org/](https://mybinder.org/)

